# Debug g1

https://github.com/bklieger-groq/g1.git

In [87]:
import os, time, json

In [88]:
import os
from openai import OpenAI
# closeai
# os.environ["OPENAI_BASE_URL"] = "https://api.openai-proxy.org/v1"
# os.environ["OPENAI_API_KEY"] = "sk-P3CPqpvL2jeUId7U1PbV7fV9J4RTqQYkZRaRu9wSyL9afpx3"
# os.environ["OPENAI_DEFAULT_MODEL"] = "gpt-4o-mini"

os.environ["OPENAI_BASE_URL"] = "http://localhost:10098/v1"
os.environ["OPENAI_API_KEY"] = "sk-not-needed"
os.environ["OPENAI_DEFAULT_MODEL"] = "default"

OPENAI_BASE_URL=os.environ.get("OPENAI_BASE_URL")
OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")
OPENAI_DEFAULT_MODEL=os.environ.get("OPENAI_DEFAULT_MODEL")
client = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)

In [89]:
OPENAI_API_KEY

'sk-not-needed'

In [90]:
import rich
from rich.console import Console
from rich.markdown import Markdown
console = Console()
def print(msg):
    console.print(Markdown(msg))

In [98]:

def make_api_call(messages, max_tokens, is_final_answer=False, custom_client=None):
    global client
    if custom_client != None:
        client = custom_client
    
    for attempt in range(3):
        try:
            if is_final_answer:
                response = client.chat.completions.create(
                    model=OPENAI_DEFAULT_MODEL,
                    messages=messages,
                    max_tokens=max_tokens,
                    temperature=0.2,
            ) 
                print(f"final {response=}")
                return response.choices[0].message.content
            else:
                response = client.chat.completions.create(
                    model=OPENAI_DEFAULT_MODEL,
                    messages=messages,
                    max_tokens=max_tokens,
                    temperature=0.2,
                    # response_format={"type": "json_object"}
                )
                print(f"{response=}")
                return json.loads(response.choices[0].message.content)
        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {"title": "Error", "content": f"Failed to generate final answer after 3 attempts. Error: {str(e)}"}
                else:
                    return {"title": "Error", "content": f"Failed to generate step after 3 attempts. Error: {str(e)}", "next_action": "final_answer"}
            time.sleep(1)  # Wait for 1 second before retrying

def generate_response(prompt, custom_client=None):
    messages = [
        {"role": "system", "content": """You are an expert AI assistant that explains your reasoning step by step. For each step, provide a title that describes what you're doing in that step, along with the content. Decide if you need another step or if you're ready to give the final answer. Respond in Markdown JSON format with 'title', 'content', and 'next_action' (either 'continue' or 'final_answer') keys. USE AS MANY REASONING STEPS AS POSSIBLE. AT LEAST 3. BE AWARE OF YOUR LIMITATIONS AS AN LLM AND WHAT YOU CAN AND CANNOT DO. IN YOUR REASONING, INCLUDE EXPLORATION OF ALTERNATIVE ANSWERS. CONSIDER YOU MAY BE WRONG, AND IF YOU ARE WRONG IN YOUR REASONING, WHERE IT WOULD BE. FULLY TEST ALL OTHER POSSIBILITIES. YOU CAN BE WRONG. WHEN YOU SAY YOU ARE RE-EXAMINING, ACTUALLY RE-EXAMINE, AND USE ANOTHER APPROACH TO DO SO. DO NOT JUST SAY YOU ARE RE-EXAMINING. USE AT LEAST 3 METHODS TO DERIVE THE ANSWER. USE BEST PRACTICES.

Example of a valid JSON response:
```json
{
    "title": "Identifying Key Information",
    "content": "To begin solving this problem, we need to carefully examine the given information and identify the crucial elements that will guide our solution process. This involves...",
    "next_action": "continue"
}```
"""},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": "Thank you! I will now think step by step following my instructions, starting at the beginning after decomposing the problem."}
    ]
    
    steps = []
    step_count = 1
    total_thinking_time = 0
    
    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 500, custom_client=custom_client)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time
        
        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))
        
        messages.append({"role": "assistant", "content": json.dumps(step_data)})
        
        if step_data['next_action'] == 'final_answer' or step_count > 25: # Maximum of 25 steps to prevent infinite thinking time. Can be adjusted.
            break
        
        step_count += 1

        # Yield after each step for Streamlit to update
        yield steps, None  # We're not yielding the total time until the end

    # Generate final answer
    messages.append({"role": "user", "content": "Please provide the final answer based solely on your reasoning above. Do not use JSON formatting. Only provide the text response without any titles or preambles. Retain any formatting as instructed by the original prompt, such as exact formatting for free response or multiple choice."})
    
    start_time = time.time()
    final_data = make_api_call(messages, 1200, is_final_answer=True, custom_client=custom_client)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time
    
    steps.append(("Final Answer", final_data, thinking_time))

    yield steps, total_thinking_time


In [99]:

# user_query = "How many Rs are in the word 'strawberry'?"
user_query = "How many letter 'r' are in the word 'strawberry'?"
for steps, total_thinking_time in generate_response(user_query):
    for i, (title, content, thinking_time) in enumerate(steps):
        # Ensure content is a string
        if not isinstance(content, str):
            content = json.dumps(content)
        if title.startswith("Final Answer"):
            print(f"### {title}")
            if '```' in content:
                parts = content.split('```')
                for index, part in enumerate(parts):
                    if index % 2 == 0:
                        print(part)
                    else:
                        if '\n' in part:
                            lang_line, code = part.split('\n', 1)
                            lang = lang_line.strip()
                        else:
                            lang = ''
                            code = part
                        print(f"```{lang}{code}```")
            else:
                print(content.replace('\n', '<br>'))
        else:
            print(content.replace('\n', '<br>'))
    
    # Only show total time when it's available at the end
    if total_thinking_time is not None:
        print(f"**Total thinking time: {total_thinking_time:.2f} seconds**")

response=ChatCompletion(id='chatcmpl-SGnI8hUcYoKZv4wG6dn9uUqVR8V1nJ3F', choices=[Choice(finish_reason='stop',      
index=0, logprobs=None, message=ChatCompletionMessage(content='json\n{\n    "title": "Counting Letters in a        
Word",\n    "content": "To determine the number of \'r\' letters in the word \'strawberry\', we will perform a     
simple letter-by-letter analysis of the word. This involves scanning through each letter and counting instances of 
the letter \'r\'. The word \'strawberry\' is spelled as follows: s-t-r-a-w-b-e-r-r-y.",\n    "next_action":        
"continue"\n}\n\njson\n{\n    "title": "Identifying the Letter \'r\' Instances",\n    "content": "Upon scanning the
word \'strawberry\', we identify the following instances of the letter \'r\': the second position (2nd \'r\'), and 
the sixth position (6th \'r\').",\n    "next_action": "continue"\n}\n\njson\n{\n    "title": "Tallying the \'r\'   
Letters",\n    "content": "By counting the identified instances of the letter \'r\' in \'strawberry\', we find     
there are a total of 2 occurrences of the letter \'r\'.",\n    "next_action": "final_answer"\n}\n\njson\n{\n       
"title": "Final Answer",\n    "content": "There are 2 letter \'r\'s in the word \'strawberry\'.",\n                
"next_action": "final_answer"\n}\n\nI have now provided a detailed, step-by-step explanation in JSON format, which 
concludes with the final answer. The reasoning involved scanning the word, identifying the letter 'r', counting its
occurrences, and then stating the final result. This process ensures accuracy and clarity in reaching the          
conclusion.<|end|>', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1727590566,   
model='default', object='chat.completion', service_tier=None, system_fingerprint=None,                             
usage=CompletionUsage(completion_tokens=402, prompt_tokens=459, total_tokens=861))

response=ChatCompletion(id='chatcmpl-VrrLFZnIWip1j2SbDo2ZnOs3NzzepjMA', choices=[Choice(finish_reason='stop',      
index=0, logprobs=None, message=ChatCompletionMessage(content='### Title: Counting the Occurrences of 'r' in       
'strawberry'\n\n#### Step 1: Identify the Target Word\nThe target word for our analysis is 'strawberry'. This is   
the word we will examine to determine the frequency of the letter 'r'.\n\n#### Step 2: Understand the Task\nOur    
task is to count how many times the letter 'r' appears in the word 'strawberry'. This is a simple character        
frequency task within a given string of text.\n\n#### Step 3: Analyze the Word\nWe will go through each letter in  
'strawberry' one by one to see if it matches the target letter 'r'.\n\n#### Step 4: Count the Occurrences\n- The   
first letter is 's'. Not 'r', so we move to the next letter.\n- The second letter is 't'. Not 'r', so we           
continue.\n- The third letter is 'r'. This matches our target letter, so we count one occurrence.\n- The fourth    
letter is 'a'. Not 'r', so we proceed.\n- The fifth letter is 'w'. Not 'r', so we keep going.\n- The sixth letter  
is 'r'. Here we have another 'r', which adds to our count, making it two occurrences so far.\n- The seventh letter 
is 'r' again. This is our third 'r', bringing the total count to three.\n- The eighth letter is 'b'. Not 'r', so we
continue.\n- The ninth letter is 'e'. Not 'r', so we move forward.\n- The tenth letter is 'r'. This is the fourth  
occurrence of 'r' in the word.\n\n#### Step 5: Summarize the Findings\nAfter analyzing each letter in 'strawberry',
we have found that the letter 'r' appears four times within the word.\n\n#### Next Action\nSince we have reached a 
conclusion, there is no need for further steps.\n\n#### Final Answer\nThe word 'strawberry' contains four instances
of the letter 'r'.\n\n```json\n{\n    "title": "Counting the Occurrences of 'r' in 'strawberry'",\n    "content":  
"After analyzing each letter in 'straw', refusal=None, role='assistant', function_call=None, tool_calls=None))],   
created=1727590579, model='default', object='chat.completion', service_tier=None, system_fingerprint=None,         
usage=CompletionUsage(completion_tokens=500, prompt_tokens=459, total_tokens=959))

response=ChatCompletion(id='chatcmpl-5F0nxYSrxKIRH2EyxxOSK5O50W2N1I4m', choices=[Choice(finish_reason='stop',      
index=0, logprobs=None, message=ChatCompletionMessage(content='### Title: Counting the Letter 'r' in               
'strawberry'\n\n#### Step 1: Identify the Target Letter\nThe first step is to identify the specific letter we are  
interested in counting within the word. In this case, the target letter is 'r'.\n\n#### Step 2: Examine the        
Word\nWe need to look at the word 'strawberry' and examine each letter to see if it matches our target letter. The 
word 'strawberry' consists of the following letters: s, t, r, a, w, b, e, r, r, r, y.\n\n#### Step 3: Count the    
Occurrences of the Target Letter\nNow that we have identified the target letter 'r' in the word 'strawberry', we   
will count how many times it appears. By examining the word, we can see that the letter 'r' appears three          
times.\n\n#### Next Action:\nSince the question has been answered with a clear and logical process, there is no    
need for further steps. The final answer is ready.\n\njson\n{\n    "title": "Counting the Letter \'r\' in          
\'strawberry\'",\n    "content": "The word \'strawberry\' contains the letter \'r\' three times.",\n               
"next_action": "final_answer"\n}\n\n\nThis JSON response provides a clear and structured explanation of how the    
count of the letter 'r' in the word 'strawberry' was determined, following the step-by-step reasoning process. The 
final answer is that there are three 'r's in the word 'strawberry'.<|end|>', refusal=None, role='assistant',       
function_call=None, tool_calls=None))], created=1727590589, model='default', object='chat.completion',             
service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=362, prompt_tokens=459,        
total_tokens=821))

final response=ChatCompletion(id='chatcmpl-YLs8A9pxNMEG2fQ0s1CKFMtuFyrZVlcZ', choices=[Choice(finish_reason='stop',
index=0, logprobs=None, message=ChatCompletionMessage(content="There are two 'r's in the word 'strawberry'.\n\nStep
1: Identify the letters in the word 'strawberry'.\nStep 2: Count the occurrences of the letter 'r'.\nStep 3:       
Confirm the count.\n\nFinal Answer: There are two 'r's in the word 'strawberry'.<|end|>", refusal=None,            
role='assistant', function_call=None, tool_calls=None))], created=1727590592, model='default',                     
object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=79,  
prompt_tokens=573, total_tokens=652))

Failed to generate step after 3 attempts. Error: Expecting value: line 1 column 1 (char 0)

Final Answer

There are two 'r's in the word 'strawberry'.Step 1: Identify the letters in the word 'strawberry'.Step 2: Count the
occurrences of the letter 'r'.Step 3: Confirm the count.Final Answer: There are two 'r's in the word               
'strawberry'.<|end|>

Total thinking time: 39.55 seconds